In [1]:
!pip install torch
!pip install torchvision
!pip install matplotlib

You are using pip version 9.0.1, however version 24.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 24.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 24.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [17]:
# !pip install pillow==5.4.1

    100% |████████████████████████████████| 2.0MB 43.6MB/s ta 0:00:01
  Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0
You are using pip version 9.0.1, however version 24.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

In [3]:
import sys
print(sys.version)

3.6.4 |Anaconda, Inc.| (default, Mar 13 2018, 01:15:57) 
[GCC 7.2.0]


In [4]:
!nvidia-smi

Sun Feb 18 14:49:38 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:00:0D.0 Off |                    0 |
| N/A   35C    P0    24W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_size = (256, 256)
print(f"device is {device}")

device is cuda


In [7]:
def read_image(image_path):
    pipeline = transforms.Compose(
        [transforms.Resize((img_size)),
         transforms.ToTensor()])

    img = Image.open(image_path).convert('RGB')
    # img = Image.open(image_path)
    img = pipeline(img).unsqueeze(0)
    return img.to(device, torch.float)

In [8]:
def save_image(tensor, image_path):
    toPIL = transforms.ToPILImage()
    img = tensor.detach().cpu().clone()
    img = img.squeeze(0)
    img = toPIL(img)
    img.save(image_path)

In [10]:
# Hyperparameters
style_img = read_image('./picasso.jpg')
content_img = read_image('./dancing.jpg')

default_content_layers = ['conv_4']
default_style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
style_weight = 1e4
content_weight = 1

In [12]:
print(style_img.shape)
print(content_img.shape)

torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])


In [13]:
class ContentLoss(torch.nn.Module):

    def __init__(self, target: torch.Tensor):
        super().__init__()
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

In [23]:
def gram(x: torch.Tensor):
    # x is a [n, c, h, w] array
    n, c, h, w = x.shape

    features = x.reshape(n * c, h * w)
    features = torch.mm(features, features.t()) / n / c / h / w
    # features = torch.mm(features, features.T) / n / c / h / w
    return features

In [15]:
class StyleLoss(torch.nn.Module):

    def __init__(self, target: torch.Tensor):
        super().__init__()
        self.target = gram(target.detach()).detach()

    def forward(self, input):
        G = gram(input)
        self.loss = F.mse_loss(G, self.target)
        return input


In [16]:
class Normalization(torch.nn.Module):

    def __init__(self, mean, std):
        super().__init__()
        self.mean = torch.tensor(mean).to(device).reshape(-1, 1, 1)
        self.std = torch.tensor(std).to(device).reshape(-1, 1, 1)

    def forward(self, img):
        return (img - self.mean) / self.std

In [17]:
def get_model_and_losses(content_img, style_img, content_layers, style_layers):
    num_loss = 0
    expected_num_loss = len(content_layers) + len(style_layers)
    content_losses = []
    style_losses = []

    model = torch.nn.Sequential(
        Normalization([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))
    cnn = models.vgg19(pretrained=True).features.to(device).eval()
    i = 0
    for layer in cnn.children():
        if isinstance(layer, torch.nn.Conv2d):
            i += 1
            name = f'conv_{i}'
        elif isinstance(layer, torch.nn.ReLU):
            name = f'relu_{i}'
            layer = torch.nn.ReLU(inplace=False)
        elif isinstance(layer, torch.nn.MaxPool2d):
            name = f'pool_{i}'
        elif isinstance(layer, torch.nn.BatchNorm2d):
            name = f'bn_{i}'
        else:
            raise RuntimeError(
                f'Unrecognized layer: {layer.__class__.__name__}')

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img)
            content_loss = ContentLoss(target)
            model.add_module(f'content_loss_{i}', content_loss)
            content_losses.append(content_loss)
            num_loss += 1

        if name in style_layers:
            target_feature = model(style_img)
            style_loss = StyleLoss(target_feature)
            model.add_module(f'style_loss_{i}', style_loss)
            style_losses.append(style_loss)
            num_loss += 1

        if num_loss >= expected_num_loss:
            break

    return model, content_losses, style_losses

In [22]:
# random_tensor = torch.rand(3, 4, 5)
# print(random_tensor.shape)

# print(random_tensor.t().shape)

torch.Size([3, 4, 5])


RuntimeError: t() expects a 2D tensor, but self is 3D

In [28]:
input_img = torch.randn(1, 3, *img_size, device=device)
model, content_losses, style_losses = get_model_and_losses(
    content_img, style_img, default_content_layers, default_style_layers)

input_img.requires_grad_(True)

# model.requires_grad_(False)

for param in model.parameters():
    param.requires_grad = False

In [30]:
optimizer = optim.LBFGS([input_img])
steps = 0
prev_loss = 0
while steps <= 1000 and prev_loss < 100:

    def closure():
        with torch.no_grad():
            input_img.clamp_(0, 1)
        global steps
        global prev_loss
        optimizer.zero_grad()
        model(input_img)
        content_loss = 0
        style_loss = 0
        for ls in content_losses:
            content_loss += ls.loss
        for ls in style_losses:
            style_loss += ls.loss
        loss = content_weight * content_loss + style_weight * style_loss
        loss.backward()
        steps += 1
        if steps % 50 == 0:
            print(f'Step {steps}:')
            print(f'Loss: {loss}')
            save_image(input_img, f'work_dirs/output_{steps}.jpg')
        prev_loss = loss
        return loss

    optimizer.step(closure)

Step 50:
Loss: 2.7176737785339355
Step 100:
Loss: 2.116436243057251
Step 150:
Loss: 1.924469232559204
Step 200:
Loss: 1.8613355159759521
Step 250:
Loss: 1.8342278003692627
Step 300:
Loss: 1.8178861141204834
Step 350:
Loss: 1.8142468929290771
Step 400:
Loss: 2.65592622756958
Step 450:
Loss: 1.8093419075012207
Step 500:
Loss: 2.366575002670288
Step 550:
Loss: 1.8306089639663696
Step 600:
Loss: 1.9394431114196777
Step 650:
Loss: 1.9141770601272583
Step 700:
Loss: 228713.078125


In [31]:
with torch.no_grad():
    input_img.clamp_(0, 1)
save_image(input_img, 'work_dirs/output.jpg')